# Python 5

## Décorateurs

Un décorateur sert à envelopper un fonction dans une autre:

```python
@dec
def f(arg1, arg2):
    pass
```
est équivalent à

```python
def f(arg1, arg2):
    pass
f = dec(f)
```

où `dec` prend une fonction comme argument et retourne une fonction.


Un petit test pour comprendre ce qui se passe :


In [2]:
def trace(f):
    def traced(*args, **kwargs):
        print ('>>')
        f(*args, **kwargs)
        print ('<<')
    return traced

@trace
def f1(truc):
    print ('truc:', truc)

@trace
def f2(x, y):
    print ('x:', x, 'y:', y)
    f1((x, y))


In [11]:
f1(2)

>>
truc: 2
<<


In [8]:
f2('bla' + 'ksjlkdqjl',[666,66])

>>
x: blaksjlkdqjl y: [666, 66]
>>
truc: ('blaksjlkdqjl', [666, 66])
<<
<<


### Exemple : mise en cache des valeurs d'une fonction récursive (*memoization*)

Permet d'automatiser le procédé, déjà illustré sur l'exemple de la suite de Fibonacci.

Sans décorateur :

In [20]:
def f(n):
    if n<2: return n
    return f(n-1)+f(n-2)

In [21]:
[f(n) for n in range(15)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

In [22]:
cache = {0:0, 1:1}

def f(n):
    try: return cache[n]
    except KeyError:
        cache[n] = f(n-1)+f(n-2)
        return cache[n]

In [23]:
f(2000)

4224696333392304878706725602341482782579852840250681098010280137314308584370130707224123599639141511088446087538909603607640194711643596029271983312598737326253555802606991585915229492453904998722256795316982874482472992263901833716778060607011615497886719879858311468870876264597369086722884023654422295243347964480139515349562972087652656069529806499841977448720155612802665404554171717881930324025204312082516817125

Avec :

In [17]:
def memoize(f):
    cache = {}
    def memoized(*args):
        try:
            return cache[args]
        except KeyError:
            result = cache[args] = f(*args)
            return result
    return memoized

In [18]:
@memoize
def fib(n):
    if n<2: return n
    return fib(n-1)+fib(n-2)

In [24]:
fib(1000)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

In [25]:
for i in range(1,11): print (fib(i*100),'\n')

354224848179261915075 

280571172992510140037611932413038677189525 

222232244629420445529739893461909967206666939096499764990979600 

176023680645013966468226945392411250770384383304492191886725992896575345044216019675 

139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125 

110433070572952242346432246767718285942590237357555606380008891875277701705731473925618404421867819924194229142447517901959200 

87470814955752846203978413017571327342367240967697381074230432592527501911290377655628227150878427331693193369109193672330777527943718169105124275 

69283081864224717136290077681328518273399124385204820718966040597691435587278383112277161967532530675374170857404743017623467220361778016172106855838975759985190398725 

54877108839480000051413673948383714443800519309123592724494953427039811201064341234954387521525390615504949092187441218246679104731442473022013980160407007017175697317900483275246652938800 

4346655768693745643568852767504

En Python 3, le module `functools` exporte un décorateur, `@lru_cache`, 
qui construit un [cache LRU](https://en.wikipedia.org/wiki/Cache_replacement_policies#Least_Recently_Used_.28LRU.29).

In [26]:
from functools import lru_cache
@lru_cache()
def fib2(n):
    if n<2: return n
    return fib2(n-1)+fib2(n-2)

In [27]:
for i in range(1,11): print (fib2(i*100),'\n')

354224848179261915075 

280571172992510140037611932413038677189525 

222232244629420445529739893461909967206666939096499764990979600 

176023680645013966468226945392411250770384383304492191886725992896575345044216019675 

139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125 

110433070572952242346432246767718285942590237357555606380008891875277701705731473925618404421867819924194229142447517901959200 

87470814955752846203978413017571327342367240967697381074230432592527501911290377655628227150878427331693193369109193672330777527943718169105124275 

69283081864224717136290077681328518273399124385204820718966040597691435587278383112277161967532530675374170857404743017623467220361778016172106855838975759985190398725 

54877108839480000051413673948383714443800519309123592724494953427039811201064341234954387521525390615504949092187441218246679104731442473022013980160407007017175697317900483275246652938800 

4346655768693745643568852767504

### Exemple : mesure du temps d'éxécution d'une fonction :

In [28]:
import time

def timeit(f):
    def timed(*args, **kw):
        ts = time.time()
        result = f(*args, **kw)
        te = time.time()
        print ('%r (%r, %r) %2.2f sec' % (f.__name__, args, kw, te-ts))
        return result
    return timed

In [29]:
@timeit
def g(t):
    print ('début')
    time.sleep(t)
    print ('fin')
    
g(3.145926535)

début
fin
'g' ((3.145926535,), {}) 3.15 sec


In [30]:
class C(object):
    @timeit
    def __init__(self):
        time.sleep(2.718281828)
        print ('Fini !')

c=C()

Fini !
'__init__' ((<__main__.C object at 0x7f88e81ba710>,), {}) 2.72 sec


### Exemple : vérifier le type d'un argument :

In [31]:
def require_int(f):
    def wrapper (arg):
        assert isinstance(arg, int)
        return f(arg)
    return wrapper

@require_int
def h(n):
    print  (n, " est un entier.")

In [32]:
h(42)

42  est un entier.


In [33]:
h(2.71828)

AssertionError: 

Les décorateurs peuvent être empilés :

In [34]:
@timeit
@require_int
def rien(n):
    time.sleep(n)
    print ('Fini')

In [35]:
rien(3)

Fini
'wrapper' ((3,), {}) 3.00 sec


In [36]:
rien(3.0)

AssertionError: 

On remarque que l'erreur est attribuée à la fonction `wrapper`. Si on n'a pas sous les yeux le code des décorateurs utilisés,
on peut chercher longtemps l'origine du problème ...

### Décorateurs avec arguments

```pytyhon
@dec(argA, argB)
def f(arg1, arg2):
    pass
```
est équivalent à
```python
def f(arg1, arg2):
    pass
f = dec(argA, argB)(f)
```

C'est donc équivalent à créer une fonction composée
`f = dec(argA, argB)(f)` 

Autrement dit, `dec(argA, argB)` doit être un décorateur.



### Exemple : ajouter un attribut à une fonction :


In [37]:
def add_attr(val):
    def decorated(f):
        f.attribute = val
        return f
    return decorated

@add_attr('Nouvel attribut')
def f():
    pass

In [38]:
f.attribute

'Nouvel attribut'

### Exemple : tester le type de la valeur retournée par une fonction :

In [39]:
def return_bool(bool_value):
    def wrapper(func):
        def wrapped(*args):
            result = func(*args)
            if result != bool_value:
                raise TypeError
            return result
        return wrapped
    return wrapper

@return_bool(True)
def always_true():
    return True

@return_bool(False)
def always_false():
    return True

In [40]:
always_true()

True

In [41]:
always_false()

TypeError: 

### Décorateurs définis par des classes

La seule contrainte sur l'objet retourné par un décorateur est
qu'il se comporte comme une fonction (*duck typing*), autrement
dit qu'il soit *callable*.

C'est le cas de toute classe possédant la méthode spéciale `__call__`.

```Python
class MyDecorator(object):
    def __init__(self, f):
    # faire quelquechose avec f ...
   def __call__(*args):
    # faire autre chose avec args
```

In [43]:
class Memoized(object):
   def __init__(self, f):
      self.f = f
      self.cache = {}
   def __call__(self, *args):
      if args in self.cache:
         return self.cache[args]
      else:
         value = self.f(*args)
         self.cache[args] = value
         return value
   def __repr__(self):
      '''Return the function's docstring.'''
      return self.f.__doc__
    
   def __str__(self): return 'toto'

In [44]:
@Memoized
def lucas(n):
    "Calcule la suite de Lucas"
    if n==0: return 2
    elif n==1: return 1
    else: return lucas(n-1)+lucas(n-2)

In [45]:
print ([lucas(i) for i in range(50)])

[2, 1, 3, 4, 7, 11, 18, 29, 47, 76, 123, 199, 322, 521, 843, 1364, 2207, 3571, 5778, 9349, 15127, 24476, 39603, 64079, 103682, 167761, 271443, 439204, 710647, 1149851, 1860498, 3010349, 4870847, 7881196, 12752043, 20633239, 33385282, 54018521, 87403803, 141422324, 228826127, 370248451, 599074578, 969323029, 1568397607, 2537720636, 4106118243, 6643838879, 10749957122, 17393796001]


In [49]:
print(fib)

<function memoize.<locals>.memoized at 0x7f88e84b28c0>


In [47]:
lucas

Calcule la suite de Lucas

In [48]:
print(lucas)

toto


### Exemple : permettre à une fonction de compter combien de fois elle a été appélée :

In [50]:
class countcalls(object):
    def __init__(self, func):
        self.__func = func
        self.__numcalls = 0
    def __call__(self, *args, **kwargs):
        self.__numcalls += 1
        return self.__func(*args, **kwargs)
    def count(self):
        return self.__numcalls

In [51]:
@countcalls
def p(): print ('*',end=' ')

@countcalls
def q(): print ('+')

for i in range(5):
    p()

for i in range(4):
    p();q()

* * * * * * +
* +
* +
* +


In [52]:
p.count(), q.count()

(9, 4)

### Méthodes de classes et méthodes statiques

On a déjà vu la différence entre les variables des classes et celles des instances :


In [54]:
class A(object):
    c = 0
    def __init__(self):
        self.c +=1

class B(object):
    c = 0
    def __init__(self):
        B.c +=1


In [55]:
a=A(); b=A()
print (a.c, b.c, A.c)

1 1 0


In [56]:
x=B(); y=B()
print (x.c, y.c, B.c)

2 2 2


Les méthodes normales sont des méthodes d'instances.
Leur premier argument doit être l'instance elle-même, conventionnellement
appelée `self`.

Il existe
aussi des méthodes de classes. 
On les définit comme les méthodes d'instance,
leur premier argument est alors la classe
elle-même, conventionnellement appelée `cls`,  puis on les
passe à la fonction `classmethod` :

In [37]:
class ASimpleClass(object):
    description = 'a simple class'
    def show_class(cls, msg):
        print ('%s: %s' % (cls.description , msg, ))
        show_class = classmethod(show_class)

C'est plus clair avec un décorateur :

In [38]:
class ASimpleClass(object):
    description = 'a simple class'
    @classmethod
    def show_class(cls, msg):
        print ('%s: %s' % (cls.description , msg, ))

Par exemple, la classe `B` qui tient un compte de ses instances pourrait s'écrire


In [39]:
class B(object):
    c = 0
    def __init__(self):
        B.c += 1

    @classmethod
    def compte_instances(cls):
        print ('instances : %d' % (cls.c, ))

Une méthode statique ne prend ni une instance ni la classe comme premier paramètre.
Elle se définit à l'aide de la fonction `staticmethod` ou du décorateur
`@staticmethod`.


In [40]:
class B(object):
    c = 0
    def __init__(self):
        B.c += 1

    @staticmethod
    def compte_instances():
        print ('instances : %d' % (B.c, ))

In [41]:
a=B(); b=B(); c=B()
B.compte_instances()

instances : 3


## Itération, itérateurs et `itertools`

Rappel :

- Syntaxe de l'itération : `for x in <quelquechose>`
- `quelquechose` peut être une liste, un tuple, une chaîne, un dictionnaire, un fichier ouvert, un ensemble ...
- Ces objets itérables possèdent une méthode spéciale `__iter__`
- On l'appelle au moyen de la fonction `iter`
- Elle retourne un itérateur auquel on peut appliquer la fonction `next`


In [58]:
ll = [1, 2, 3, 4, 5]
it = ll.__iter__()
next(it)

1

In [59]:
next(it)

2

Normalement, on écrit plûtôt

In [63]:
ll = [1, 2, 3, 4, 5]
it = iter(ll)
it

In [64]:
list(it)

[1, 2, 3, 4, 5]

In [65]:
list(it)

[]

On peut définir des classes qui supportent l'itération : il suffit
d'implémenter les méthodes `__iter__` et `__next__`.

Les boucles longues sont peu efficaces et sont une des principales causes
de lenteur en Python. 

Pour des boucles sur les entiers, en Python 2, on utilisera `xrange` (un générateur
écrit directement en C)
plutôt que `range`.  

Pour des itérations plus compliquées, on pourra utiliser le module `itertools`,
qui propose des version optimisées d'opérations courantes, et de nombreuses fonctionnalités
commodes.

### On peut chainer des itérateurs :



In [18]:
from itertools import *

it=chain(range(5),range(5,-1,-1))
list(it)

[0, 1, 2, 3, 4, 5, 4, 3, 2, 1, 0]

### ou les tricoter ... 


In [19]:
it=zip(range(5),range(5,-1,-1))
list(it)

[(0, 5), (1, 4), (2, 3), (3, 2), (4, 1)]

### Compteurs

`count(start,step=1)` engendre les entiers à partir de `start`
avec les pas `step`. 

La fonction `islice(iterable,[start],stop,[step])`
remplace `iterable[start:stop:step]`:


In [20]:
it=islice(count(5),7,17,2)
list(it)

[12, 14, 16, 18, 20]

### Prooduit cartésien

Avec les mots binaires de 5 digits, on pourrait faire


In [21]:
['01' for i in range(5)]

['01', '01', '01', '01', '01']

In [22]:
list(product('01','01','01'))

[('0', '0', '0'),
 ('0', '0', '1'),
 ('0', '1', '0'),
 ('0', '1', '1'),
 ('1', '0', '0'),
 ('1', '0', '1'),
 ('1', '1', '0'),
 ('1', '1', '1')]

In [23]:
list(map(bin,range(16,24)))

['0b10000',
 '0b10001',
 '0b10010',
 '0b10011',
 '0b10100',
 '0b10101',
 '0b10110',
 '0b10111']

In [24]:
words = product(*['01' for i in range(5)])
ll=islice(words,16,24)

list(ll)

[('1', '0', '0', '0', '0'),
 ('1', '0', '0', '0', '1'),
 ('1', '0', '0', '1', '0'),
 ('1', '0', '0', '1', '1'),
 ('1', '0', '1', '0', '0'),
 ('1', '0', '1', '0', '1'),
 ('1', '0', '1', '1', '0'),
 ('1', '0', '1', '1', '1')]

La fonction `product` renvoie
le produit cartésien (les tuples) d'un nombre arbitraire d'itérables.

On peut s'en servir pour construire les mots de longueur donnée
sur un alphabet, comme dans ce craqueur de mots de passe basique :

In [25]:
from crypt import crypt

def words(alphabet,length):
    return product(*[alphabet for i in range(length)])

def crack(hash_, salt, alphabet,length):
    ww = words(alphabet,length)
    for w in ww:
        p = crypt(''.join(w),salt)
        if p == hash_:
            print ('Password found: ', ''.join(w))
            return ''.join(w)


In [26]:
from string import ascii_lowercase
pw = 'gabu'
slt ='XY'
h = crypt(pw,slt)
h

'XYhc/C.XeLVM6'

In [27]:
crack(h,'XY',ascii_lowercase,4)

Password found:  gabu


'gabu'

Ce n'est pas très efficace (!) mais c'est simple ...


### tee
La fonction `tee(iterateur,n=2)` retourne $n$ copies
identiques de l'itérateur :

In [28]:
it = islice(count(), 5)
i1, i2, i3 = tee(it,3)
[(next(i1), next(i2),next(i3)) for i in range(5)]

[(0, 0, 0), (1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4)]

In [29]:
list(it) # entièrement consommé

[]

### starmap
La fonction `starmap` fonctionne comme `map`, mais
calcule `f(*i)` :

In [30]:
it = zip('abcd', range(1,5))
ff = starmap(lambda x,y: x*y,it)
list(ff)

['a', 'bb', 'ccc', 'dddd']

### cycle
La fonction `cycle` répète indéfiniment un itérateur fini :


In [86]:
it = cycle('bla')
[next(it) for i in range(12)]

['b', 'l', 'a', 'b', 'l', 'a', 'b', 'l', 'a', 'b', 'l', 'a']

### repeat
La fonction `repeat` fait ce qu'on imagine :

In [87]:
it=repeat('bla',4)
list(it)

['bla', 'bla', 'bla', 'bla']

On l'utilise en combinaison avec `map` ou `zip` :

In [60]:
it = zip(range(5), repeat(2), 'abcd')
list(it)

[(0, 2, 'a'), (1, 2, 'b'), (2, 2, 'c'), (3, 2, 'd')]

### Filtrage 
Le filtrage s'effectue au moyen des fonctions 
`dropwhile, takewhile, filter, filterfalse` :

In [61]:
it=takewhile(lambda x:x*x<100, count())
list(it)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [62]:
it=dropwhile(lambda x:x<0, range(-6,5))
list(it)

[0, 1, 2, 3, 4]

In [63]:
it=filter(lambda x:x%2, range(10))
list(it)

[1, 3, 5, 7, 9]

In [64]:
it=filterfalse(lambda x:x%2, range(10))
list(it)

[0, 2, 4, 6, 8]

### groupby

Plus complexe : `groupby` construit un itérateur qui renvoie
les clés et groupes consécutifs d'un itérable.La clé `key`
est une fonction qui calcule une valeur sur chaque élément. Par défaut,
c'est l'identité. 

On pourra donc écrire

In [65]:
ll = [1,2,2,2,1,1,4,4,5,5,2,2,1,1,3,3,1,1]
it=groupby(ll)
[k for k,g in it]

[1, 2, 1, 4, 5, 2, 1, 3, 1]

Le résultat complet serait

In [66]:
it=groupby(ll)
[(k, list(g))  for k,g in it]

[(1, [1]),
 (2, [2, 2, 2]),
 (1, [1, 1]),
 (4, [4, 4]),
 (5, [5, 5]),
 (2, [2, 2]),
 (1, [1, 1]),
 (3, [3, 3]),
 (1, [1, 1])]

In [67]:
ll = [('a',1), ('b',2), ('c',2), ('d',1), ('e',2), ('f',1)]
it=groupby(ll, lambda x: x[1])
[(k, list(map(lambda y:y[0],g)) ) for k,g in it]

[(1, ['a']), (2, ['b', 'c']), (1, ['d']), (2, ['e']), (1, ['f'])]

Finalement, on dispose de quelques fonctions combinatoires basiques :

In [68]:
it=combinations('abcd',2)
list(it)

[('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')]

In [69]:
it=combinations_with_replacement('abcd',2)
list(it)

[('a', 'a'),
 ('a', 'b'),
 ('a', 'c'),
 ('a', 'd'),
 ('b', 'b'),
 ('b', 'c'),
 ('b', 'd'),
 ('c', 'c'),
 ('c', 'd'),
 ('d', 'd')]

In [89]:
it=permutations(range(1,4))
list(it)

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

## Autres optimisations

Le module `operator` propose des fonctions optimisées
pour remplacer les opérateurs standards de Python (ex. `add(x,y)`).

Le module `collections` fournit des structures de données
hautes performances pour remplacer  `dict, list, set, tuple` :
`namedtuple(), deque, Counter, OrderedDict, defaultdict`.

Le module `array` fournit des tableaux optimisés pour 
des types de données basiques (caractères, entiers, flottants ...)


## Le module `ctypes`

Il permet d'utiliser des bibliothèques partagées, avec des types de
données compatibles au C.

In [91]:
from ctypes import *
libc =  cdll.LoadLibrary("libc.so.6")

In [92]:
printf=libc.printf
printf(b"%s\n", b"Hello world!") # noter le b"" nécessaire pour python 3

13

```
...
[I 08:28:51.990 NotebookApp] Saving file at /python3_M1_2020-5.ipynb
Hello world!
[I 08:38:52.001 NotebookApp] Saving file at /python3_M1_2020-5.ipynb
...
```

Le résultat s'est affiché sur le terminal dans lequel on a lancé l'interface graphique ...

Dans un terminal, on obtiendrait
```Python
>>> from ctypes import *
>>> libc =  cdll.LoadLibrary("libc.so.6")
>>> printf=libc.printf
>>> printf(b"%s\n", b"Hello world!")
Hello world!
13
>>> 
```

In [93]:
 print (libc.time(None))

1604481104


Pour de longues itérations, on peut écrire en C la fonction critique
et la compiler sous forme *dll/shared object*

Un petit test de performances :
```C
/* rien.c
 compiler avec
   gcc -Wall -fPIC -c rien.c
   gcc -shared -Wl,-soname,librien.so.1 -o librien.so.1.0   *.o
*/

int rien(int n){
    int i=0;
    while (1==1) {
        i++;
        if(i>n){ return(i); }
    }
}
```

In [94]:
from ctypes import *
cdll.LoadLibrary("./librien.so.1.0")
librien=CDLL("./librien.so.1.0")

def rien(n):
    i=0
    while 1==1:
        i+=1
        if i>n: return i
        
from time import time
print ("Avec le C :")
a=time()
librien.rien(10000000)
print (time()-a)
print ("En pur Python :")
a=time()
rien(10000000)
print (time()-a)

Avec le C :
0.021549701690673828
En pur Python :
0.5683567523956299


## Cython

Pour étendre Python avec du code C ou C++, il vaut mieux
utiliser [Cython](http://cython.org)

- Cython est un sur-langage de Python (basé sur Pyrex) avec les types de données du C 
- Il possède un compilateur optimisé
- Presque tout code Python est aussi du code Cython valide
- En déclarant les types, on obtient du code très efficace

Après avoir installé Cython, on peut reprendre l'exemple précédent. On crée un fichier `rien.pyx`

```Python
# rien.pyx`
def rien(int n):
    cdef int i=0
    while 1==1:
        i+=1
        if i>n: return i
```

Le code est le même, à ceci près que les types `int`
de `n` et `i` ont été déclarés.

Pour compiler, il faut, dans
le même répertoire, un fichier `setup.py` structuré ainsi :

```Python
# setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup( ext_modules = cythonize("rien.pyx") )
```

On compile avec la commande

`python setup.py build_ext --inplace`

On peut alors importer `rien` comme un module ordinaire


In [95]:
from time import time
import rien

def pyrien(n):
    i=0
    while 1==1:
        i+=1
        if i>n: return i

print ("Avec le C :")
a=time()
rien.rien(10000000)
print (time()-a)
print ("En pur Python :")
a=time()
pyrien(10000000)
print (time()-a)

Avec le C :
0.0011870861053466797
En pur Python :
0.5563614368438721


In [96]:
0.5563614368438721/0.0011870861053466797

468.6782486443061

468 fois plus rapide ici, donc. Notons au passage que si on n'avait pas déclaré les types, l'effet aurait été beaucoup moins bon (un facteur 2).